In [1]:
import pandas as pd
import numpy as np
import requests
import datetime as dt
from bs4 import BeautifulSoup
import ast
from utils._funcs import creating_club_dataframe,Transfmkt

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


##### Club name and id

##### Funcs

In [2]:
lst1 = ['05/06', '21/22', '10/11', '14/15']
lst_sep = []
# xx/yy - 21/22

# for lst in lst1:
#     lst_sep.append([lst])


def get_all_seasons(temp,temp_limit):
    all = []
    all.append(temp)
    if temp == temp_limit:
        pass
    else:
        seasons_n = int(temp_limit[3:]) - int(temp[:2])
        for season in range(1,seasons_n):
            if float(temp[3:]) < 9.0:
                temp = str(temp[3:]) + str('/0'+str(int(float(temp[3:])+1)))
                all.append(temp)
            else:
                temp = str(temp[3:]) + str('/'+str(int(float(temp[3:])+1)))
                all.append(temp)
    return all


get_all_seasons('00/01','21/22')

for l in lst1:
    lst_sep.append(get_all_seasons(l,'21/22'))


def get_saison(list):
    saison = []
    for item in list:
        saison.append(str('20'+item[:2]))

    return saison

def get_saison_perf_links(saisons_list,perf_links):
    all_perf_season_links = []
    for pflink in perf_links:
        perf_season_links = []
        for saison in saisons_list:
            perf_season_links.append(str(pflink + '/plus/0?saison='+saison+'&verein=&liga=&wettbewerb=&pos=&trainer_id='))
        all_perf_season_links.append(perf_season_links)
    
    return all_perf_season_links


saisons = []
for list in lst_sep:
    saisons.append(get_saison(list))

lst_final = []
for list in saisons:
    lst_final.append(get_saison_perf_links(list,lst_test))

lst_final = str(lst_final)[1:-1]
lst_final = eval(lst_final)[0]

# lst_final = get_saison_perf_links((list for list in saisons),lst_test)
lst_final

NameError: name 'lst_test' is not defined

## Transfermkt Class

In [2]:
"""
To make the request to the page we have to inform the
website that we are a browser and that is why we
use the headers variable
"""
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
transf = Transfmkt('se-palmeiras','1023', headers)
transf.creating_lists()

In [3]:
transf.get_players_seasons()
df = transf.get_players_transfers_clubs_links()

In [4]:
df_test = df.groupby('data').first().reset_index()
df_test

,data,link_transferencia
0,2005,/sc-corinthians/transfers/verein/199/saison_id...
1,2006,/clube-do-remo/transfers/verein/10997/saison_i...
2,2007,/sc-corinthians/transfers/verein/199/saison_id...
3,2008,/america-fc-rn-/transfers/verein/1751/saison_i...
4,2009,/associacao-portuguesa-de-desportos-sp-/transf...
5,2011,/athletico-paranaense/transfers/verein/679/sai...
6,2017,/se-palmeiras/transfers/verein/1023/saison_id/...


### Gráfico interativo

In [2]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_binary
import datetime as dt
import pandas as pd

# Opening the connection and grabbing the page
my_url = 'https://www.google.com/webhp?hl=en'
option = Options()
option.headless = True

In [3]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(800, 800))  
display.start()
driver = webdriver.Chrome(executable_path='/home/gabriel/.local/bin/chromedriver',options=option)
driver.get(my_url)
driver.maximize_window()

FileNotFoundError: [Errno 2] No such file or directory: 'Xvfb'